# Preprocessamento e criação do modelo

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from fuzzywuzzy import process 

In [18]:
df = pd.read_csv('./data/bronze/data.csv')

## Tratamento de valores nulos

In [ ]:
def fill_na(df):
    for col in df.columns:
        if df[col].dtype == 'object':  
            moda = df[col].mode()
            if not moda.empty:  
                moda = moda[0]
            df[col] = df[col].fillna(moda)  
        else:  
            median = df[col].median()
            df[col] = df[col].fillna(median)


In [22]:
new_df = df
fill_na(new_df)
new_df.head()

,profissao,tempoprofissao,renda,tiporesidencia,escolaridade,score,idade,dependentes,estadocivil,produto,valorsolicitado,valortotalbem,classe
0,Cientista de Dados,24,58660.0,Outros,Ens.Médio,MuitoBom,57.0,0,Solteiro,VoyageRoamer,84623.0,350000.0,bom
1,Empresário,21,46557.0,Outros,Ens.Médio,MuitoBom,37.0,2,Víuvo,EcoPrestige,126855.0,500000.0,bom
2,Dentista,13,43939.0,Própria,Ens.Médio,Bom,22.0,0,Casado,DoubleDuty,127151.0,320000.0,ruim
3,Engenheiro,10,37262.0,Própria,Superior,Baixo,35.0,0,Divorciado,AgileXplorer,28767.0,250000.0,bom
4,Contador,6,52606.0,Própria,PósouMais,Justo,26.0,0,Casado,TrailConqueror,199564.0,400000.0,ruim


## Tratamento da coluna profissao

In [23]:
profissoes_validas = ['Advogado','Arquiteto','Cientista de Dados',
                      'Contador','Dentista','Empresário','Engenheiro',
                      'Médico','Programador']

In [25]:
def fix_write_errors(df,coluna, lista_valida):
    for i, valor in enumerate(df[coluna]):
        valor_str = str(valor) if pd.notnull(valor) else valor
        
        if valor_str not in lista_valida and pd.notnull(valor_str):
            fix = process.extractOne(valor_str, lista_valida)[0]
            df.at[i,coluna] = fix


In [26]:
fix_write_errors(new_df,'profissao', profissoes_validas)
new_df.head()

,profissao,tempoprofissao,renda,tiporesidencia,escolaridade,score,idade,dependentes,estadocivil,produto,valorsolicitado,valortotalbem,classe
0,Cientista de Dados,24,58660.0,Outros,Ens.Médio,MuitoBom,57.0,0,Solteiro,VoyageRoamer,84623.0,350000.0,bom
1,Empresário,21,46557.0,Outros,Ens.Médio,MuitoBom,37.0,2,Víuvo,EcoPrestige,126855.0,500000.0,bom
2,Dentista,13,43939.0,Própria,Ens.Médio,Bom,22.0,0,Casado,DoubleDuty,127151.0,320000.0,ruim
3,Engenheiro,10,37262.0,Própria,Superior,Baixo,35.0,0,Divorciado,AgileXplorer,28767.0,250000.0,bom
4,Contador,6,52606.0,Própria,PósouMais,Justo,26.0,0,Casado,TrailConqueror,199564.0,400000.0,ruim


## Tratamento de outliers

In [27]:
def fix_outliers(df,coluna, min, max):
    median = df[(df[coluna] >= min) & (df[coluna] <= max)][coluna].median()
    df[coluna] = df[coluna].apply(lambda x: median if x < min or x > max else x)
    return df

In [28]:
new_df = fix_outliers(new_df,'idade', 0, 110)
new_df = fix_outliers(new_df,'tempoprofissao', 0, 70)

In [29]:
new_df.head()

,profissao,tempoprofissao,renda,tiporesidencia,escolaridade,score,idade,dependentes,estadocivil,produto,valorsolicitado,valortotalbem,classe
0,Cientista de Dados,24.0,58660.0,Outros,Ens.Médio,MuitoBom,57.0,0,Solteiro,VoyageRoamer,84623.0,350000.0,bom
1,Empresário,21.0,46557.0,Outros,Ens.Médio,MuitoBom,37.0,2,Víuvo,EcoPrestige,126855.0,500000.0,bom
2,Dentista,13.0,43939.0,Própria,Ens.Médio,Bom,22.0,0,Casado,DoubleDuty,127151.0,320000.0,ruim
3,Engenheiro,10.0,37262.0,Própria,Superior,Baixo,35.0,0,Divorciado,AgileXplorer,28767.0,250000.0,bom
4,Contador,6.0,52606.0,Própria,PósouMais,Justo,26.0,0,Casado,TrailConqueror,199564.0,400000.0,ruim
